# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [21]:
# Upgrade sklearn to its latest version
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /local/tarciso/anaconda3/envs/udacity-ds-p2/lib/python3.7/site-packages (0.23.1)


In [22]:
import sklearn
print(sklearn.__version__)

0.22.1


In [1]:
# import libraries

#Basic DS libs
import numpy as np
import pandas as pd

#Helper Libs
import re
from sqlalchemy import create_engine
import joblib

#NLTK
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split

#Vectorizers/Transformers
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

#Models
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Evaluation Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss, zero_one_loss, accuracy_score, precision_score, recall_score

#Gensim
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

#Glove
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

#Doc2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
import multiprocessing

[nltk_data] Downloading package punkt to /home/tarciso/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tarciso/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tarciso/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import sklearn
sklearn.__version__

'0.23.1'

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
messages_df = pd.read_sql_table(con=engine, table_name='Message')
categories_df = pd.read_sql_table(con=engine, table_name='CorpusWide')

In [4]:
# create categories_list data with a list of the categories for each message
categories = pd.read_csv('../data/disaster_categories.csv')
categories['categories_list'] = categories['categories'].apply(lambda x: re.sub(r"[_a-z]+-0[;]?", "", x)) \
                                    .apply(lambda x: re.sub(r"-1", "", x)) \
                                    .apply(lambda x: re.sub(r";$", "", x))

In [5]:
messages_df

,message_id,message,original,genre,num_words
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12
...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,21
26140,30262,A suitable candidate has been selected and OCH...,None,news,22
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,23
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,31


In [6]:
categories_df

,message_id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26140,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
categories

,id,categories,categories_list
0,2,related-1;request-0;offer-0;aid_related-0;medi...,related
1,7,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;other_aid;weather_related;...
2,8,related-1;request-0;offer-0;aid_related-0;medi...,related
3,9,related-1;request-1;offer-0;aid_related-1;medi...,related;request;aid_related;medical_products;o...
4,12,related-1;request-0;offer-0;aid_related-0;medi...,related
...,...,...,...
26243,30261,related-0;request-0;offer-0;aid_related-0;medi...,
26244,30262,related-0;request-0;offer-0;aid_related-0;medi...,
26245,30263,related-1;request-0;offer-0;aid_related-0;medi...,related
26246,30264,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;military


### 2 - Design and Apply Tokenization Strategies

#### 2.1 - Write down tokenization functions

In [8]:
def tokenize_to_list(text, lemmatizer):
    '''
    INPUT
    text - text string to be tokenized
    lemmatizer - lemmatizer object to be used to process text tokens
    
    OUTPUT
    A list of tokens extracted from the input text
    
    This function receives raw text as input a pre-processes it for NLP analysis, removing punctuation and
    special characters, normalizing case and removing extra spaces, as well as removing stop words and 
    applying lemmatization
    '''
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tokens

In [9]:
def tokenize_to_str(text, lemmatizer):
    '''
    INPUT
    text - text string to be tokenized
    lemmatizer - lemmatizer object to be used to process text tokens
    
    OUTPUT
    A string with the tokens extracted from the input text concatenated by spaces
    
    This function receives raw text as input a pre-processes it for NLP analysis, removing punctuation and
    special characters, normalizing case and removing extra spaces, as well as removing stop words and 
    applying lemmatization
    '''
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]
    #Return tokens list as a string joined by whitespaces
    clean_tokens_str = ' '.join(clean_tokens)

    return clean_tokens_str

In [10]:
lemmatizer = WordNetLemmatizer()
tokenize_to_list(messages_df.message[0],lemmatizer)

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [11]:
tokenize_to_str(messages_df.message[0],lemmatizer)

'weather update cold front cuba could pas haiti'

#### 2.2 - Generate tokenized messages table for further analysis

In [38]:
# %%time
# lemmatizer = WordNetLemmatizer()
# messages_df['tokens_list'] = messages_df.message.apply(lambda x: tokenize_to_list(x, lemmatizer))

In [39]:
%%time
messages_df['tokens_str'] = messages_df.message.apply(lambda x: tokenize_to_str(x, lemmatizer))

CPU times: user 1min 20s, sys: 6.67 s, total: 1min 26s
Wall time: 1min 26s


In [40]:
messages_df

,message_id,message,original,genre,num_words,tokens_str
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13,weather update cold front cuba could pas haiti
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9,hurricane
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6,looking someone name
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13,un report leogane 80 90 destroyed hospital st ...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12,say west side haiti rest country today tonight
...,...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,21,training demonstrated enhance micronutrient pr...
26140,30262,A suitable candidate has been selected and OCH...,None,news,22,suitable candidate selected ocha jakarta curre...
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,23,proshika operating cox bazar municipality 5 un...
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,31,2 000 woman protesting conduct election tearga...


In [41]:
messages_tokens = messages_df[['message_id','tokens_str']]

In [42]:
messages_tokens.to_sql('MessageTokens', engine, index=False, if_exists='replace')

In [43]:
#messages_categories = messages_tokens.merge(categories_df, on='message_id')

In [44]:
#messages_categories.head()

#### 2.4 - Find n-grams and save to DB

In [45]:
# def get_ngrams_freqs(messages_array, n=1):
#     vec = CountVectorizer(ngram_range=(n, n)).fit(messages_array)
#     bag_of_words = vec.transform(messages_array)
#     word_count = bag_of_words.sum(axis=0)
#     words_freq = [(word, n, word_count[0, idx]) for word, idx in vec.vocabulary_.items()]
#     words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
#     words_freq_df = pd.DataFrame(data = words_freq, columns = ['ngram','n','count'])
#     return words_freq_df

In [46]:
# unigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str)
# bigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str, n=2)
# trigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str, n=3)
# ngrams_freqs = pd.concat([unigrams_freqs, bigrams_freqs, trigrams_freqs])

In [47]:
# ngrams_freqs

In [48]:
# ngrams_freqs.to_sql('NGramsFreqs', engine, index=False, if_exists='replace')

### 3 - Prepare Data for Train/Test Pipeline
- Extract X and Y from datasets
- Split data into train and test sets
- Train pipeline

In [12]:
messages_tokens = pd.read_sql_table(con=engine, table_name='MessageTokens')

In [13]:
messages_tokens

,message_id,tokens_str
0,2,weather update cold front cuba could pas haiti
1,7,hurricane
2,8,looking someone name
3,9,un report leogane 80 90 destroyed hospital st ...
4,12,say west side haiti rest country today tonight
...,...,...
26139,30261,training demonstrated enhance micronutrient pr...
26140,30262,suitable candidate selected ocha jakarta curre...
26141,30263,proshika operating cox bazar municipality 5 un...
26142,30264,2 000 woman protesting conduct election tearga...


In [51]:
# prepare data for ML Pipeline
X = messages_df.message.values
X_tokenized = messages_tokens.tokens_str.values
Y_df = categories_df.drop(['message_id', 'message', 'original', 'genre'], axis=1)
Y = Y_df.values
category_columns = Y_df.columns

In [52]:
Y_df.sum()

related                   20031
request                    4453
offer                       118
aid_related               10822
medical_help               2078
medical_products           1310
search_and_rescue           723
security                    471
military                    857
water                      1666
food                       2911
shelter                    2303
clothing                    403
money                       602
missing_people              298
refugees                    873
death                      1190
other_aid                  3434
infrastructure_related     1701
transport                  1196
buildings                  1327
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1147
weather_related            7272
floods                     2142
storm                      2437
fire                        282
earthqua

In [53]:
categories_df

,message_id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26139,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26140,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26141,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26142,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [55]:
X_tokenized

array(['weather update cold front cuba could pas haiti', 'hurricane',
       'looking someone name', ...,
       'proshika operating cox bazar municipality 5 union ramu chokoria assessment 5 kg rice 1 5 kg lentil 700 family',
       '2 000 woman protesting conduct election teargassed tried converge local electoral commission office southern oil city port harcourt',
       'radical shift thinking came result meeting recognizing hiv aid core humanitarian crisis identifying crisis function hiv aid pandemic'],
      dtype=object)

In [56]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [57]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X_tokenized, Y, test_size=0.25, random_state=199)

In [59]:
X_train

array(['crude charm charismatic dt member thanks video santiago talk twice',
       'tidal wave triggered earthquake indonesia sunday swept vast swath coastline including india indonesia malaysia maldives somalia sri lanka thailand',
       'earthquake happened 4h34 evening night', ...,
       'earthquake day haiti google possibly leaving china',
       'matter debated previously non aligned movement foreign minister firmly stated right humanitarian intervention',
       'dumbonyc oh rt endtwist coned power station dumbo flooding nopower sandy'],
      dtype=object)

In [60]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [61]:
print(X_train.shape, y_train.shape)

(19608,) (19608, 35)


In [62]:
y_train.sum(axis=0)

array([15027,  3310,    88,  8115,  1570,   982,   538,   354,   669,
        1253,  2161,  1731,   293,   461,   224,   659,   898,  2558,
        1276,   879,  1000,   410,   117,   223,    89,   240,   855,
        5479,  1597,  1818,   214,  1875,   399,  1024,  3791])

___

## Feature Engineering

##### 4 - Design Transformer to Tokenize sentences before passing on to other transformers

In [ ]:
class Tokenizer(BaseEstimator):

    def __init__(self, output_str=True):
        self.output_str = output_str
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([tokenize_to_str(tokens_str, lemmatizer) if self.output_str else 
                         tokenize_to_list(tokens_str, lemmatizer) for tokens_str in X])
            

In [ ]:
# %%time
# msgTokenizer = Tokenizer().fit(X, None)
# msgTokenizer.transform(X)

### 4 - Create a Mean/TF-IDF word vector from a locally trained Word2Vec model

#### 4.1 - Train a Word2Vec model using messages text

In [ ]:
class Message(object):
    """An interator that yields messages tokens (lists of str)."""
    
    def __init__(self, messages_df, sample_size=-1):
        self.messages_df = messages_df
        self.sample_size = sample_size

    def __iter__(self):
        messages = self.messages_df
        
        #Sample dataset if specified
        if self.sample_size > 0:
            messages = messages.sample(self.sample_size)
        
        #Yeld tokenized message joined by whitespaces
        for token_str in messages.tokens_str:
            yield token_str.split()

In [ ]:
messages_tokens.tokens_str

In [ ]:
%%time

sentences = Message(messages_df=messages_tokens)
local_w2v_model_full = gensim.models.Word2Vec(sentences=sentences)
local_w2v_model_full.save("messages-word2vec-full.model")

In [ ]:
w2v_model_full = gensim.models.Word2Vec.load("messages-word2vec-full.model")

In [ ]:
# look up generated word vocab
for i, word in enumerate(w2v_model_full.wv.vocab):
    if i == 10:
        break
    print(word)

In [ ]:
# look up generated word vectors
w2v_model_full.wv['earthquake']

#### 4.2 - Implement Vector Aggregator

Inspecting variables

In [ ]:
# len(X)

In [ ]:
# X[0]

In [ ]:
# X.shape[0]

Initial implementation of aggregators

In [ ]:
from collections import defaultdict

class MeanEmbeddingVectorizer(BaseEstimator):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [ ]:
# #Testing defaultdict functionality
# test_dict = defaultdict(lambda: 1, [("Hello" , 7), ("hi" , 10), ("there" , 45),("at" , 23),("this" , 77)])
# test_dict['Hi']

In [ ]:
class TfidfEmbeddingVectorizer(BaseEstimator):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
        self.word_weights = None
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

Test aggregators using a Pipeline

In [ ]:
# %%time

# w2v_mean_pip = Pipeline([
#     ('features',FeatureUnion([
#         ('w2v_mean', MeanEmbeddingVectorizer(w2v_model_full)),
#         ('w2v_tfidf', TfidfEmbeddingVectorizer(w2v_model_full))
#     ])),    
#     ('clf', MultiOutputClassifier(GaussianNB()))
# ])

# w2v_mean_pip.fit(X_train, y_train)

# y_pred = w2v_mean_pip.predict(X_test)

# for category_idx in range(y_pred.shape[1]):
#     print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

In [ ]:
# category_idx=1
# c_report = classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], 
#                       labels=[0,1], 
#                       target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1'],
#                       output_dict=True)
# c_report

In [ ]:
# print(c_report.keys())

In [ ]:
# w2v_mean_pip.score(X_test, y_test)

In [ ]:
# y_pred[0]

In [ ]:
y_pred.shape

In [ ]:
y_test[0]

In [ ]:
y_test.shape

In [ ]:
sum(y_pred[0] == y_test[0])/y_pred[0].shape[0]

In [ ]:
sample_test = y_test[0]
sample_pred = y_pred[0]

In [ ]:
sample_test == 1

In [ ]:
sample_pred == 1

In [ ]:
(sample_test == 1) == (sample_pred == 1)

In [ ]:
#Computing TPR manually
tp_array = (sample_test == 1) & (sample_pred == 1)
tp = tp_array.sum()
tpr = tp/sample_test.shape[0]
print(tp)
print(tpr)
tp_array

In [ ]:
3/36

In [ ]:
#Computing FPR manually
fp_array = (sample_test == 0) & (sample_pred == 1)
fp = fp_array.sum()
fpr = fp/sample_test.shape[0]
print(fp)
print(fpr)
fp_array

In [ ]:
#Computing FNR manually
fn_array = (sample_test == 1) & (sample_pred == 0)
fn = fn_array.sum()
fnr = fn/sample_test.shape[0]
print(fn)
print(fnr)
fn_array

In [ ]:
#Computing Precision manually
print(tp/(tp+fp))

In [ ]:
#Computing Recall manually
print(tp/(tp+fn))

In [ ]:
example_test = np.append(np.repeat(1, 4),np.repeat(0,32))
example_test

In [ ]:
example_pred = np.repeat(0, 36)
example_pred

In [ ]:
from sklearn.metrics import f1_score

print("Hamming Loss = ", hamming_loss(example_test,example_pred))
print("Zero-One Loss = ", zero_one_loss(example_test,example_pred))
print("Jaccard Score = ", jaccard_score(example_test,example_pred))
print("Accuracy = ", accuracy_score(example_test,example_pred))
print("Recall = ", recall_score(example_test,example_pred))
print("Precision = ", precision_score(example_test,example_pred))
print("F1-Score = ", f1_score(example_test,example_pred))
print("Jaccard Score Micro = ", jaccard_score(example_test,example_pred, average='micro'))
print("Recall Micro = ", recall_score(example_test,example_pred, average='micro'))
print("Precision Micro = ", precision_score(example_test,example_pred, average='micro'))
print("F1-Score Micro = ", f1_score(example_test,example_pred, average='micro'))

In [ ]:
print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Recall = ", recall_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Precision = ", precision_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score
print(mean_absolute_error(y_test, y_pred, multioutput='raw_values'))
print(mean_absolute_error(y_test, y_pred))
(y_test == y_pred).mean()
print(r2_score(y_test, y_pred))
print(r2_score(y_test, y_pred, multioutput='variance_weighted'))
print(explained_variance_score(y_test, y_pred))
print(explained_variance_score(y_test, y_pred, multioutput='variance_weighted'))

#### 4.3 - Create Estimator to both train and average w2v on corpus data

Trying W2VTransformer - Gensim's sklearn Transformer

In [ ]:
from gensim.sklearn_api import W2VTransformer
from gensim.test.utils import common_texts

model = W2VTransformer(size=10, min_count=1, seed=1)

model.fit(common_texts)
          
model.transform(['graph', 'system'])

In [ ]:
common_texts

In [ ]:
messages_df.head(5).tokens_str.str.split().values

In [ ]:
messages_df.head(5).tokens_str

In [ ]:
w2v_model = W2VTransformer()
w2v_model.fit(messages_df.tokens_str.str.split().values)

In [ ]:
word = messages_df.tokens_str[0].split()[1]
print(word)
w2v_model.transform(word)

In [ ]:
test_model = gensim.models.Word2Vec(messages_df.tokens_str.str.split().values)

In [ ]:
for i, word in enumerate(test_model.wv.vocab):
    if i == 10:
        break
    print(word)

In [ ]:
test_model.wv['earthquake']

Re-implement aggregators now with training capability

Test aggregators using a Pipeline

In [ ]:
# %%time

# from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import classification_report

# w2v_mean_pip = Pipeline([
#     ('features',FeatureUnion([
#         ('w2v_mean', MeanEmbeddingTrainVectorizer(word2vec_model=w2v_model_full)),
#         ('w2v_mean_train', MeanEmbeddingTrainVectorizer(num_dims=50)),
#         ('w2v_tfidf', TfidfEmbeddingTrainVectorizer(word2vec_model=w2v_model_full)),
#         ('w2v_tfidf_train', TfidfEmbeddingTrainVectorizer(num_dims=100)),
#     ])),
#     ('clf', MultiOutputClassifier(GaussianNB()))
# ])

# w2v_mean_pip.fit(X_train, y_train)

# y_pred = w2v_mean_pip.predict(X_test)

# for category_idx in range(y_pred.shape[1]):
#     print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

### 5. Use Glove pre-trained model to generate feature vectors

Glove pre-trained vectors can be downloaded here: https://nlp.stanford.edu/projects/glove/

%%time

# Code which can be used to convert from Glove default format to Gensim W2V format

glove_file = 'glove-pretrained/glove.6B.50d.txt'
tmp_file = 'glove-pretrained/glove.6B.50d_word2vec.txt'

#_ = glove2word2vec(glove_file, tmp_file)

glove_model = KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
%%time

from gensim.models import KeyedVectors

glove_50d_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.50d_word2vec.txt', binary=False)

In [ ]:
%%time

glove_100d_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.100d_word2vec.txt', binary=False)

In [ ]:
%%time

glove_300d_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.300d_word2vec.txt', binary=False)

##### Use a Pipeline to train the Word2Vec estimator

Test Aggregators with Glove using a Pipeline

In [ ]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('features',FeatureUnion([
        ('glove_mean', MeanEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ])),    
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("y_pred sum = ",y_pred.sum())
print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))

In [ ]:
#from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import average_precision_score

# For each class
# precision = dict()
# recall = dict()
# average_precision = dict()
# for i in range(len(category_columns)):
#     precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_pred[:, i])
    #average_precision[i] = average_precision_score(y_test[:, i], y_pred[:, i])

# A "micro-average": quantifying score on all classes jointly
# precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
#     y_score.ravel())
# average_precision["micro"] = average_precision_score(Y_test, y_score,
#                                                      average="micro")
# print('Average precision score, micro-averaged over all classes: {0:0.2f}'
#       .format(average_precision["micro"]))

In [ ]:
from sklearn.metrics import hamming_loss, zero_one_loss, jaccard_score, accuracy_score, auc, recall_score, precision_score

print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Recall = ", recall_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Precision = ", precision_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))

In [ ]:
y_test[0] == y_pred[0]

In [ ]:
y_test[0]

In [ ]:
y_pred[0]

In [ ]:
6536*36

In [ ]:
print(np.argmax(y_pred, axis=1).shape)
print(y_pred.shape)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
train_foo = [['sci-fi', 'thriller'],['comedy'],['sci-fi', 'thriller'],['comedy']]
mlb = MultiLabelBinarizer()
mlb_label_train = mlb.fit_transform(train_foo)
mlb_label_train

In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
Y_df[Y_df['related'] == 2]

### 6. Use Doc2Vec to generate feature vectors

# Checking the number of words in the corpus
messages_df['message'].apply(lambda x: len(x.split(' '))).sum()

Try Doc2Vec step by step

In [ ]:
def read_messages(messages, tokens_only=False):
    for index, tokens_str in np.ndenumerate(messages):
        tokens = tokens_str.split()
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            tags = index
            yield gensim.models.doc2vec.TaggedDocument(tokens, tags)

train_corpus = list(read_messages(X_train))
test_corpus = list(read_messages(X_test, tokens_only=True))

In [ ]:
train_corpus[:10]

In [ ]:
test_corpus[:10]

In [ ]:
d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=5, epochs=10)

In [ ]:
d2v_model.build_vocab(train_corpus)

In [ ]:
%%time 

d2v_model.train(train_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

#### Create a Doc2Vec Estimator/Transformer 

In [ ]:
class Doc2VecTransformer(BaseEstimator):

    def __init__(self, vector_size=100, epochs=20):
        self.epochs = epochs
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() - 1
        self.d2v_model = None

    def fit(self, X, y):
        tagged_x = [TaggedDocument(tokens_str.split(), [index]) for index, tokens_str in np.ndenumerate(X)]
        self.d2v_model = Doc2Vec(vector_size=self.vector_size, workers=self.workers, epochs=self.epochs)
        self.d2v_model.build_vocab(tagged_x)
        self.d2v_model.train(tagged_x, total_examples=self.d2v_model.corpus_count, 
                             epochs=self.epochs)

        return self

    def transform(self, X):
        return np.asmatrix(np.array([self.d2v_model.infer_vector(tokens_str.split())
                                     for tokens_str in X]))

In [ ]:
%%time

doc2vec_trf = Doc2VecTransformer(epochs=1)
doc2vec_trf.fit(X_train, y_train)

In [ ]:
X_test[0].split()

In [ ]:
inferred_vector = doc2vec_trf.d2v_model.infer_vector(X_test[0].split())
print(inferred_vector.shape)
print(type(inferred_vector))
print(inferred_vector)

inferred_vector2 = np.asmatrix(np.array([doc2vec_trf.d2v_model.infer_vector(X_test[0].split())]))
print(inferred_vector2.shape)
print(type(inferred_vector2))
print(inferred_vector2)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
doc2vec_features = doc2vec_trf.transform(X_test)
print(doc2vec_features.shape)
doc2vec_features

Test Doc2Vec using a Pipeline

In [ ]:
%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("y_pred sum = ",y_pred.sum())

### 7. Extra Feature - Distance Between Message Word Vector to Categories Word Vectors 

We will use Glove as our space vectorizer as it is trained on a broad corpus and thus able to better describe texts / words

First, let's build the embeddings for the categories

In [ ]:
category_columns

In [ ]:
categories_tokens = np.array([np.array(cat.split('_')) for cat in category_columns])

In [ ]:
categories_tokens

In [ ]:
glove_300d_w2v_mean = MeanEmbeddingTrainVectorizer(glove_300d_w2v)

In [ ]:
glove_300d_w2v_mean.fit(categories_tokens, None)

In [ ]:
cat_tokens_embeddings = glove_300d_w2v_mean.transform(categories_tokens)

In [ ]:
print(cat_tokens_embeddings.shape)
print(cat_tokens_embeddings)

In [ ]:
cat_tokens_embeddings[0]

In [ ]:
test_msgs_embeddings = glove_300d_w2v_mean.transform(X_test)

In [ ]:
print(test_msgs_embeddings.shape)
print(test_msgs_embeddings)

In [ ]:
test_msgs_embeddings[0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

print(test_msgs_embeddings[:10].shape)
print(cat_tokens_embeddings.shape)
print(cosine_similarity(test_msgs_embeddings[:10],cat_tokens_embeddings).shape)

#### Create a CategoriesSimilarity Transformer 

Test the transformer manually using test data

In [ ]:
# cats_sim_transf = CategoriesSimilarity(categories_tokens, glove_50d_w2v)

In [ ]:
# cats_sim_transf.fit(X_test, None)

In [ ]:
# cats_sim_transf.transform(X_test)

Test CategoriesSimilarity feature using a Pipeline

In [ ]:
%%time

from sklearn.pipeline import FeatureUnion

cats_sim_pip = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_300d_w2v, cat_tokens_embeddings)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

cats_sim_pip.fit(X_train, y_train)

y_pred = cats_sim_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("Score = ", cats_sim_pip.score(X_test, y_test))

# accuracy
print("Accuracy = ",accuracy_score(y_test,y_pred))
print("\n")

### 8. NLP Feature Selection using Sklearn Pipelines

#### 8.1 - All features together

In [ ]:
%%time

from sklearn.pipeline import FeatureUnion

all_feats = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

all_feats.fit(X_train, y_train)

y_pred = all_feats.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

In [ ]:
score = all_feats.score(X_test, y_test)

In [ ]:
score

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize_to_list(lemmatizer=lemmatizer), ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=199), n_jobs=-1))
])

In [ ]:
# pipeline.fit(X_train, y_train)

#### Build pipelines for different combinations of features and classifiers

In [ ]:
from sklearn import svm

local_w2v_tfidf_svc = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

local_w2v_tfidf_rf = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

glove_tfidf_svc = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

glove_tfidf_rf = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

doc2vec_svc = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

doc2vec_rf = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

all_feats_svc = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

all_feats_rf = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pipelines = [local_w2v_tfidf_svc, local_w2v_tfidf_rf, glove_tfidf_svc, glove_tfidf_rf, 
             doc2vec_svc, doc2vec_rf, all_feats_svc, all_feats_rf]

grid_dict = {0 : 'Local Word2Vec - TF-IDF - Linear SVC',
             1 : 'Local Word2Vec - TF-IDF - Random Forest',
             2 : 'Glove 300d - TF-IDF - Linear SVC',
             3 : 'Glove 300d - TF-IDF - Random Forest',
             4 : 'Doc2Vec - Linear SVC',
             5 : 'Doc2Vec - Random Forest',
             6 : 'All Features - Linear SVC',
             7 : 'All Features - Random Forest',}

In [ ]:
# param_grid = [
#     {'features' : [MeanEmbeddingTrainVectorizer(),TfidfEmbeddingTrainVectorizer()],
#      'features__word2vec_model' : [None,glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
#      'features__num_dims' : [50,100,300]},
#     {'features' : [Doc2VecTransformer()],
#      'features__vector_size' : [50,100,300]},
#     {'features' : [CategoriesSimilarity(glove_50d_w2v,categories_tokens)],
#      'features__word2vec_model' : [glove_50d_w2v,glove_100d_w2v,glove_300d_w2v]},
#     {'features' : [MeanEmbeddingTrainVectorizer(),TfidfEmbeddingTrainVectorizer()],
#      'features__word2vec_model' : [None,glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
#      'features__num_dims' : [50,100,300]},
#     {'classifier' : [MultiOutputClassifier(LogisticRegression(random_state=199))],
#      'classifier__estimator__penalty' : ['l1', 'l2'],
#      'classifier__estimator__C' : [0.1, 1, 3],
#      'classifier__estimator__solver' : ['liblinear']},
#     {'classifier' : [MultiOutputClassifier(RandomForestClassifier(random_state=199))],
#     'classifier__estimator__n_estimators' : [10,50,250],
#     'classifier__estimator__max_depth' : [10, 50, 100],
#     'classifier__estimator__min_samples_split' : [1, 5, 10]}
# ]

In [ ]:
# generic_pip = Pipeline([
#     ('features',TfidfEmbeddingTrainVectorizer()),
#     ('classifier',MultiOutputClassifier(RandomForestClassifier(random_state=199)))
# ])

In [ ]:
# MeanEmbeddingTrainVectorizer().get_params().keys()

In [ ]:
# param_grid = [
#     {'features' : [TfidfEmbeddingTrainVectorizer()],
#      'features__word2vec_model' : [None,glove_50d_w2v],
#      'features__num_dims' : [50]}]

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

# #score = make_scorer(mean_absolute_error, greater_is_better=False)
# jobs = -1
# score = 'f1_micro'
# def_cv = 3

# generic_gs = GridSearchCV(estimator=generic_pip,
#             param_grid=param_grid,
#             scoring=score,
#             cv=def_cv,
#             n_jobs=jobs,
#             verbose=True)  

# # randomized_gs = RandomizedSearchCV(estimator=generic_pip,
# #             param_distributions=param_grid,
# #             scoring=score,
# #             cv=def_cv,
# #             n_jobs=jobs,
# #             verbose=True,
# #             random_state=199,
# #             n_iter=2)  

In [ ]:
# %%time

# grids = [generic_gs]

# grid_dict = {
#     0 : 'Generic Grid',
# }

# print('Performing Grid Search...')
# # Fit grid search
# generic_gs.fit(X_train, y_train)
# # Best params
# print('Best params: %s' % generic_gs.best_params_)
# # Best training data accuracy
# print('Best training score: %.3f' % generic_gs.best_score_)
# # Predict on test data with best params
# test_score = generic_gs.score(X_test, y_test)
# # Test data accuracy of model with best params
# print('Test set score for best params: %.3f ' % test_score)

# # Track Grid Search results
# gs_results_df = pd.DataFrame(generic_gs.cv_results_)
# gs_results_df['best_model_test_score'] = test_score

#### Run a RandomizedSearchCV for each featureset

In [ ]:
class MeanEmbeddingTrainVectorizer(BaseEstimator):

    def __init__(self, word2vec_model=None, num_dims=100):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = num_dims
            self.workers = multiprocessing.cpu_count() - 1
            
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.vector_size
            
        print(self.num_dims)
            
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X, size=self.num_dims, 
                                                         workers=self.workers)
        
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [ ]:
class TfidfEmbeddingTrainVectorizer(BaseEstimator):
    
    def __init__(self, word2vec_model=None, num_dims=100):
        self.word2vec_model = word2vec_model
        self.num_dims = num_dims
        
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.workers_ = multiprocessing.cpu_count() - 1
            self.word2vec_model = gensim.models.Word2Vec(X, size=self.num_dims, 
                                                         workers=self.workers_)
        self.num_dims = self.word2vec_model.vector_size
            
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights_ = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights_[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

class CategoriesSimilarity(BaseEstimator):
    
    def __init__(self, categories_tokens, word2vec_model=None, num_dims=100):
        self.categories_tokens = categories_tokens
        self.word2vec_model = word2vec_model    
        self.num_dims = num_dims
        
    def compute_mean_embeddings(self, tokens_array):    
        mean_embeddings = np.empty([tokens_array.shape[0],self.num_dims])
        
        for i in range(tokens_array.shape[0]):
            doc_tokens = tokens_array[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings
                    
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.workers_ = multiprocessing.cpu_count() - 1
            self.word2vec_model = gensim.models.Word2Vec(X, size=self.num_dims, 
                                                         workers=self.workers_)
        self.num_dims = self.word2vec_model.vector_size        
        self.categories_vectors_ = self.compute_mean_embeddings(self.categories_tokens)
        return self 

    def transform(self, X):
        mean_embeddings = self.compute_mean_embeddings(X)
        cats_similarities = cosine_similarity(mean_embeddings, self.categories_vectors_)
            
        return cats_similarities

##### Define Pipelines for each featureset

In [ ]:
import nlp_estimators

In [ ]:
#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split

#Vectorizers/Transformers
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

#Models
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Evaluation Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss, zero_one_loss, accuracy_score, precision_score, recall_score

In [ ]:
pip_local_w2v = Pipeline([
    ('local_w2v', nlp_estimators.TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# pip_glove = Pipeline([
#     ('glove', nlp_estimators.TfidfEmbeddingTrainVectorizer(glove_50d_w2v)),
#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
# ])

pip_doc2vec = Pipeline([
    ('doc2vec', nlp_estimators.Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# pip_cats_sim = Pipeline([
#     ('cats_sim', nlp_estimators.CategoriesSimilarity(categories_tokens=categories_tokens)),
#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
# ])

##### Define Grid Params for each featureset / classifier

In [ ]:
params_options_models_simple = [MultiOutputClassifier(RandomForestClassifier(random_state=199,
                                                        n_estimators=50,
                                                        max_depth=100,
                                                        min_samples_split=5)),
                                            MultiOutputClassifier(LogisticRegression(random_state=199,
                                                                                    solver='liblinear',
                                                                                    C=1,
                                                                                    penalty='l2'))]

params_options_local_w2v_simple = {'local_w2v__num_dims' : [50,100],
                                   'clf' : params_options_models_simple}

params_options_local_w2v = {'local_w2v__num_dims' : [50,100,300],
                            'clf' : params_options_models_simple}

params_options_glove = {'glove__word2vec_model' : [glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
                            'clf' : params_options_models_simple}

params_options_doc2vec = {'doc2vec__vector_size' : [50,100,300],
                            'clf' : params_options_models_simple}

params_options_cat_sim = {'cats_sim__word2vec_model' : [glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
                            'clf' : params_options_models_simple}

params_options_rf = {'clf' : [MultiOutputClassifier(RandomForestClassifier(random_state=199))],
                     'clf__estimator__n_estimators' : [50,250],
                     'clf__estimator__max_depth' : [50,100],
                     'clf__estimator__min_samples_split' : [2, 5]}

params_options_lg = {'clf' : [MultiOutputClassifier(LogisticRegression(random_state=199))],
                     'clf__estimator__penalty' : ['l1', 'l2'],
                     'clf__estimator__C' : [0.1, 1, 3],
                     'clf__estimator__solver' : ['liblinear']}

params_options_empty = {}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

#score = make_scorer(mean_absolute_error, greater_is_better=False)
jobs = -1
score = 'f1_micro'
def_cv = 3
verbosity_level=10

gs_local_w2v = GridSearchCV(estimator=pip_local_w2v,
            param_grid=params_options_local_w2v,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level)  

gs_glove = GridSearchCV(estimator=pip_glove,
            param_grid=params_options_glove,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level) 

gs_doc2vec = GridSearchCV(estimator=pip_doc2vec,
            param_grid=params_options_doc2vec,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level)

gs_cats_sim = GridSearchCV(estimator=pip_cats_sim,
            param_grid=params_options_cat_sim,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level) 

randomized_gs = RandomizedSearchCV(estimator=pip_local_w2v,
            param_distributions=params_options_local_w2v,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level,
            random_state=199,
            n_iter=5)

In [ ]:
%%time

grids = [gs_local_w2v, gs_glove, gs_doc2vec, gs_cats_sim]

grid_dict = {
    0 : 'Feature: Local W2V',
    1 : 'Feature: Glove',
    2 : 'Feature: Doc2Vec',
    3 : 'Feature: Category Similarity'
}

# grids = [gs_all_feats_best_params]

# grid_dict = {
#     0 : 'Feature: All Features with Best Params',
# }

# grids = [gs_local_w2v]

# grid_dict = {
#     0 : 'Feature: Local W2V',
# }

print('Performing model optimizations...')
best_score = 0.0
best_clf = 0
best_gs = ''
overall_results_df = pd.DataFrame()
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    gs_results_df['param_set_order'] = np.arange(len(gs_results_df))
    
    overall_results_df = pd.concat([overall_results_df, gs_results_df])
    
    # Track best (highest test accuracy) model
    if test_score > best_score:
        best_score = test_score
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grid_dict[best_clf])

In [ ]:
overall_results_df

In [ ]:
results_feature_sets = overall_results_df

In [ ]:
results_feature_sets.to_csv('f1micro_score_per_featset.csv', index=False)

In [ ]:
gs_local_w2v.best_estimator_

In [ ]:
gs_glove.best_estimator_

In [ ]:
gs_doc2vec.best_estimator_

In [ ]:
gs_cats_sim.best_estimator_

In [ ]:
test = pd.read_csv('f1micro_score_per_featset.csv')
test[test['rank_test_score'] == 1]

In [ ]:
#Test All Features together using their best params from GridSearch

In [ ]:
pip_all_feats = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v', TfidfEmbeddingTrainVectorizer(num_dims=300)),
        ('glove', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer(vector_size=50)),
        ('cats_sim', CategoriesSimilarity(categories_tokens=categories_tokens,word2vec_model=glove_300d_w2v))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [ ]:
params_options_all_feats = {'clf' : params_options_models_simple}

In [ ]:
gs_all_feats_best_params = GridSearchCV(estimator=pip_all_feats,
                                  param_grid=params_options_all_feats,
                                 scoring=score,
                                 cv=def_cv,
                                 n_jobs=jobs,
            verbose=verbosity_level)

In [ ]:
grids = [gs_all_feats_best_params]

grid_dict = {
    0 : 'Feature: All Features with Best Params',
}

print('Performing model optimizations...')
all_feats_results = pd.DataFrame()
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    gs_results_df['param_set_order'] = np.arange(len(gs_results_df))
    
    all_feats_results = pd.concat([all_feats_results, gs_results_df])

In [ ]:
all_feats_results

In [ ]:
all_feats_results.to_csv('f1micro_score_all_feats.csv')

#### Final Decision between models

In [ ]:
pip_local_w2v_final = Pipeline([
    ('local_w2v', TfidfEmbeddingTrainVectorizer(num_dims=300)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pip_all_feats_final = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v', TfidfEmbeddingTrainVectorizer(num_dims=300)),
        ('glove', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer(vector_size=50)),
        ('cats_sim', CategoriesSimilarity(categories_tokens=categories_tokens,word2vec_model=glove_300d_w2v))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [ ]:
params_options_rf = {'clf' : [MultiOutputClassifier(RandomForestClassifier(random_state=199))],
                     'clf__estimator__n_estimators' : [25,50,100],
                     'clf__estimator__max_depth' : [100],
                     'clf__estimator__min_samples_split' : [2, 5]}

In [ ]:
gs_local_w2v_final = GridSearchCV(estimator=pip_local_w2v_final,
            param_grid=params_options_rf,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level) 

gs_all_feats_final = GridSearchCV(estimator=pip_all_feats_final,
                                  param_grid=params_options_rf,
                                 scoring=score,
                                 cv=def_cv,
                                 n_jobs=jobs,
            verbose=verbosity_level)

In [ ]:
grids_final = [gs_local_w2v_final]

grid_final_dict = {
    0 : 'Feature: Local W2V with Best Params',
    1 : 'Feature: All Features with Best Params'
}

print('Performing model optimizations...')
final_results = pd.DataFrame()
for idx, gs in enumerate(grids_final):
    print('\nEstimator: %s' % grid_final_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_final_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    gs_results_df['param_set_order'] = np.arange(len(gs_results_df))
    
    final_results = pd.concat([final_results, gs_results_df])

In [ ]:
final_results_local_w2v = final_results
final_results_local_w2v.to_csv('f1_score_local_w2v_final.csv', index=False)

In [ ]:
param_grid_ = [
    {'features' : [MeanEmbeddingTrainVectorizer(),TfidfEmbeddingTrainVectorizer()],
     'features__word2vec_model' : [None,glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
     'features__num_dims' : [50,100,300]},
    {'features' : [Doc2VecTransformer()],
     'features__vector_size' : [50,100,300]},
    {'features' : [CategoriesSimilarity(glove_50d_w2v,categories_tokens)],
     'features__word2vec_model' : [glove_50d_w2v,glove_100d_w2v,glove_300d_w2v]},
    {'features' : [MeanEmbeddingTrainVectorizer(),TfidfEmbeddingTrainVectorizer()],
     'features__word2vec_model' : [None,glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
     'features__num_dims' : [50,100,300]},
    {'classifier' : [MultiOutputClassifier(LogisticRegression(random_state=199))],
     'classifier__estimator__penalty' : ['l1', 'l2'],
     'classifier__estimator__C' : [0.1, 1, 3],
     'classifier__estimator__solver' : ['liblinear']},
    {'classifier' : [MultiOutputClassifier(RandomForestClassifier(random_state=199))],
    'classifier__estimator__n_estimators' : [10,50,250],
    'classifier__estimator__max_depth' : [10, 50, 100],
    'classifier__estimator__min_samples_split' : [1, 5, 10]}
]

In [ ]:
from sklearn import svm
from sklearn.metrics import make_scorer, mean_absolute_error

pip_local_w2v_tfidf_naive = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_local_w2v_tfidf_rf = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pip_glove_tfidf_naive = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_glove_tfidf_rf = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pip_glove_tfidf_lg = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(LogisticRegression(random_state=199)))
])

pip_doc2vec_naive = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_doc2vec_rf = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pip_cats_sim_naive = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_cats_sim_rf = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pip_all_feats_naive = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_all_feats_rf = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

In [ ]:
grid_params_empty = [{}]

# grid_params_rf = [{'clf__criterion': ['gini', 'entropy'],
#         'clf__min_samples_leaf': param_range,
#         'clf__max_depth': param_range,
#         'clf__min_samples_split': param_range[1:]}]

# grid_params_svm = [{'clf__kernel': ['linear', 'rbf'], 
#         'clf__C': param_range}]

# Construct grid searches
jobs = -1

#score = make_scorer(mean_absolute_error, greater_is_better=False)
score = 'f1_micro'

def_cv = 3

gs_local_w2v_tfidf_naive = GridSearchCV(estimator=pip_local_w2v_tfidf_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)  

gs_local_w2v_tfidf_rf = GridSearchCV(estimator=pip_local_w2v_tfidf_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)

gs_glove_tfidf_naive = GridSearchCV(estimator=pip_glove_tfidf_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs) 

gs_glove_tfidf_rf = GridSearchCV(estimator=pip_glove_tfidf_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)

gs_glove_tfidf_lg = GridSearchCV(estimator=pip_glove_tfidf_lg,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)


gs_doc2vec_naive = GridSearchCV(estimator=pip_doc2vec_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)

gs_doc2vec_rf = GridSearchCV(estimator=pip_doc2vec_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)

gs_cats_sim_naive = GridSearchCV(estimator=pip_cats_sim_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs) 

gs_cats_sim_rf = GridSearchCV(estimator=pip_cats_sim_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs) 

gs_all_feats_naive = GridSearchCV(estimator=pip_all_feats_naive,
                                  param_grid=grid_params_empty,
                                 scoring=score,
                                 cv=def_cv,
                                 n_jobs=jobs)

gs_all_feats_rf = GridSearchCV(estimator=pip_all_feats_rf,
                                  param_grid=grid_params_empty,
                                 scoring=score,
                                 cv=def_cv,
                                 n_jobs=jobs)

# grids = [gs_local_w2v_tfidf_rf, gs_glove_tfidf_rf, 
#          gs_doc2vec_rf, gs_cats_sim_rf,
#          gs_all_feats_rf]

# grid_dict = {
#     0 : 'Local Word2Vec - TF-IDF - Random Forest',
#     1 : 'Glove 300d - TF-IDF - Random Forest',
#     2 : 'Doc2Vec - Random Forest',
#     3 : 'Categories Similarity - Random Forest',
#     4 : 'All Features - Random Forest'
# }

# grids = [gs_local_w2v_tfidf_naive, gs_glove_tfidf_naive, 
#          gs_doc2vec_naive, gs_cats_sim_naive,
#          gs_all_feats_naive]

# grid_dict = {
#     0 : 'Local Word2Vec - TF-IDF - Naive Bayes',
#     1 : 'Glove 300d - TF-IDF - Naive Bayes',
#     2 : 'Doc2Vec - Naive Bayes',
#     3 : 'Categories Similarity - Naive Bayes',
#     4 : 'All Features - Naive Bayes'
# }

grids = [gs_glove_tfidf_lg]

grid_dict = {
    0 : 'Glove 300d - TF-IDF - Logistic Regression',
}

In [ ]:
%%time

print('Performing model optimizations...')
best_score = 0.0
best_clf = 0
best_gs = ''
overall_results_df = pd.DataFrame()
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    
    overall_results_df = pd.concat([overall_results_df, gs_results_df])
    
    # Track best (highest test accuracy) model
    if test_score > best_score:
        best_score = test_score
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grid_dict[best_clf])

In [ ]:
overall_results_df

In [ ]:
results_lg_f1micro_score = overall_results_df

In [ ]:
results_lg_f1micro_score.to_csv('naive_f1micro_score.csv', index=False)

In [ ]:
# Save best grid search pipeline to file
dump_file = 'best_gs_pipeline.pkl'
joblib.dump(best_gs, dump_file, compress=1)
print('\nSaved %s grid search pipeline to file: %s' % (grid_dict[best_clf], dump_file))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
X_train.shape

In [ ]:
X_train

In [ ]:
# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
print(X_test.shape)

In [ ]:
print(y_pred.shape)
y_pred

In [ ]:
y_pred[:,0]

In [ ]:
y_test

In [ ]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params().keys()

In [ ]:
parameters = {
        'multi_clf__estimator__n_estimators': [20,50],
        'multi_clf__estimator__max_depth': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred = cv.predict(X_test)

In [ ]:
cv.best_params_

In [ ]:
for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# Output a pickle file for the model
joblib.dump(cv, 'classifier.pkl') 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.